In [15]:
import pandas as pd

In [16]:
df = pd.read_csv('/Phishing Dataset.csv', names = ["label", "message"])
df.head()

,label,message
0,legitimate,http://www.crestonwood.com/router.php
1,phishing,http://shadetreetechnology.com/V4/validation/a...
2,phishing,https://support-appleld.com.secureupdate.duila...
3,legitimate,http://rgipt.ac.in
4,legitimate,http://www.iracing.com/tracks/gateway-motorspo...


In [17]:
df.rename(columns = {'label':'Category', 'message':'Message'}, inplace = True)
df.head()

,Category,Message
0,legitimate,http://www.crestonwood.com/router.php
1,phishing,http://shadetreetechnology.com/V4/validation/a...
2,phishing,https://support-appleld.com.secureupdate.duila...
3,legitimate,http://rgipt.ac.in
4,legitimate,http://www.iracing.com/tracks/gateway-motorspo...


In [18]:
df['phishing']=df['Category'].apply(lambda x: 1 if x=='phishing' else 0)
df.head()

,Category,Message,phishing
0,legitimate,http://www.crestonwood.com/router.php,0
1,phishing,http://shadetreetechnology.com/V4/validation/a...,1
2,phishing,https://support-appleld.com.secureupdate.duila...,1
3,legitimate,http://rgipt.ac.in,0
4,legitimate,http://www.iracing.com/tracks/gateway-motorspo...,0


In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['Message'],df['phishing'], stratify=df['phishing'])
X_train.head(4)

10954                   https://www.1and1.com/domain-check
1766     https://www.marsbahis272.com/?btag=655160_8F21...
2328     http://www.sloaneandhyde.com/imm/new2015/pvali...
681      http://stefaninicompany.net/webadmin/login.php...
Name: Message, dtype: object

In [20]:
!pip install -q -U tensorflow-text

In [21]:
!pip install -q -U tf-models-official

In [22]:
!pip install -U tfds-nightly

In [23]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [24]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

In [25]:
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [26]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(32, activation='relu')(l)
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)
# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size = 32)

Epoch 1/100
268/268 [==============================] - 93s 319ms/step - loss: 0.5988 - accuracy: 0.6709
Epoch 2/100
268/268 [==============================] - 86s 323ms/step - loss: 0.5419 - accuracy: 0.7264
Epoch 3/100
268/268 [==============================] - 87s 325ms/step - loss: 0.5325 - accuracy: 0.7308
Epoch 4/100
268/268 [==============================] - 87s 325ms/step - loss: 0.5111 - accuracy: 0.7452
Epoch 5/100
268/268 [==============================] - 87s 325ms/step - loss: 0.5049 - accuracy: 0.7545
Epoch 6/100
268/268 [==============================] - 87s 326ms/step - loss: 0.4970 - accuracy: 0.7576
Epoch 7/100
268/268 [==============================] - 87s 326ms/step - loss: 0.5025 - accuracy: 0.7531
Epoch 8/100
268/268 [==============================] - 87s 325ms/step - loss: 0.4922 - accuracy: 0.7582
Epoch 9/100
268/268 [==============================] - 87s 325ms/step - loss: 0.4867 - accuracy: 0.7629
Epoch 10/100
268/268 [==============================] - 87s 325m

In [14]:
scores = model.evaluate(X_test, y_test)
print('\nAccuracy {0:.2f}%'.format(scores[1]*100))

90/90 [==============================] - 30s 322ms/step - loss: 0.3987 - accuracy: 0.8359

Accuracy 83.59%
